## Aggregate rasters
This is the workflow for aggregating predictor rasters to the coarser resolutions used for analysis (96.5 km, 193 km, and 385.9 km). Note- many of these aggregations use up a lot of memory that R doesn't efficiently dump. You may need to restart R in between aggregating some rasters to free up memory.

In [ ]:
library(raster)
library(rnaturalearth)
library(here)
library(tictoc)
library(tidyverse)
library(furrr)

out_path <- here("data", "climate_agg")
res_list <- list(high = 96.5, medium = 193, low = 385.9)

Create template rasters to resample predictors and filter genetic data with. 

Template resolutions:  
* 96.5 km (1 degree at 30 deg latitude) {**template_high**}
* 193 km (2 degrees) {**template_medium**}
* 385.9 km (4 degrees) {**template_low**}  


*Note that I am not aggregating predictor rasters to the low resolution. After exploring the genetic data, we determined that a low resolution is not compatible with our analyses. I'm leaving the template here because we used it to filter the genetic data*

In [5]:
crs <- "+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +datum=WGS84 +ellps=WGS84 +units=m +no_defs"

# read in a world shape file. Might make this able to use countries as well
world_shp <- rnaturalearth::ne_coastline(scale = "small")
  
# convert the coordinate system to behrmann
world_shp <- sp::spTransform(world_shp, crs)
  
# turn it into a raster
world_raster <- raster::raster(world_shp)


# transform world raster to high resolution
template_high <- world_raster
res(template_high) <- 96500


# transform world raster to medium resolution
template_medium <- world_raster
res(template_medium) <- 193000

# transform world raster to low resolution
template_low <- world_raster
res(template_low) <- 385900

# write to data/templates/ folder
if (!file.exists(here("data", "templates", "template_high.tif"))){
    writeRaster(template_high, filename = here("data", "templates", "template_high.tif"))
}

if (!file.exists(here("data", "templates", "template_medium.tif"))){
    writeRaster(template_medium, filename = here("data", "templates", "template_medium.tif"))
}

if (!file.exists(here("data", "templates", "template_low.tif"))){
    writeRaster(template_medium, filename = here("data", "templates", "template_low.tif"))
}

This function follows these steps:  
* reproject raster to Behrmann equal-area projection
* resample with bilinear interpolation to the coarse resolution

This process is slow, but results in rasters with the precise desired resolution in km. Alternatives result in imprecise resolutions, where for example a 96.5 km resolution specification, the resolution is actually 96.48628 km or 95 km. 

In [6]:
# function to resample rasters to any equal area grid of the three resolutions we're considering.


# crs: define the crs you want to project to. It must be an equal area projection. The default crs is behrmann equal area. 
# x: a raster or raster stack
# km: km resolution you want (area will be km x km)
# is_categorical: if is_categorical is true, using "ngb" interpolation
resample_equal_area <- function(x, 
                                crs = "+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +datum=WGS84 +ellps=WGS84 +units=m +no_defs", 
                                km,
                                is_categorical = FALSE) {
  
  # project raster to new coordinate system (this has no values- we put values back in later)
  x_ext <- raster::projectExtent(x, crs)
  
  # make the resolution square since we're interested in an equal area projection
  raster::res(x_ext) <- raster::xres(x_ext)
  
  # add original values to our projection. We can't take shortcuts like doing this instead of doing projectExtent first because it removes cells at the extreme latitudes.
  x_repro <- raster::projectRaster(x, x_ext)
  
  if (km == 96.5) {
      end_raster = template_high
  } else if (km == 193) {
      end_raster = template_medium
  } else if (km == 385.9) {
      end_raster = template_low
  } else stop("Incorrect resolution specification. Must be either 96.5 or 193")
  
  # resample our raster of interest to the beginning raster  
  if (is_categorical) {
      m <- "ngb"
  } else m <- "bilinear"
  
  
  x_resampled <- raster::resample(x_repro, end_raster, method = m) 

  
  return(x_resampled)
}


In [7]:
# function to write aggregated rasters to file, using a specified prefix for the file and layer names
write_raster <- function (x, prefix) {
    names(x) <- paste0(prefix, "_", names(x))
    out_file <- file.path(out_path, paste0(names(x), ".tif"))
    writeRaster(x, filename = out_file, overwrite = TRUE)
}

## Current climate
From [paleoclim.org](http://www.paleoclim.org/), based on [CHELSA](http://chelsa-climate.org/bioclim/). 2.5 arcmin resolution.

In [10]:
current_path <- dir(here("data", "climate_raw", "current_bioclim"), pattern = "*.tif$", full.names = TRUE)
current_rast_list <- purrr::map(current_path, raster)

#### 96.5 km (high) resolution

In [76]:
tic()
current_agg_high <- purrr::map(current_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(current_agg_high, ~write_raster(.x, prefix = "current_high")))

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto,

894.467 sec elapsed


#### 193 km (Medium) Resolution

In [95]:
tic()
current_agg_medium <- purrr::map(current_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(current_agg_medium, ~write_raster(.x, prefix = "current_medium")))

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto,

840.896 sec elapsed


## Terrain
Elevation raster from [worldclim.org](worldclim.org) (based on SRTM elevation data). 2.5 arcmin resolution. I used the `raster::terrain()` function to calculate:  
* slope
* aspect
* Topographic Position Index (TPI)
* Terrain Ruggedness Index (TRI)
* roughness

The terrain calculation script is located here: `R/calc_terrain.R`

In [83]:
terrain_path <- dir(here("data", "climate_raw", "terrain"), pattern = "*.tif$", full.names = TRUE)
terrain_rast_list <- purrr::map(terrain_path, raster)

#### 96.5 km (high) resolution

In [84]:
tic()
terrain_agg_high <- purrr::map(terrain_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(terrain_agg_high, ~write_raster(.x, prefix = "terrain_high")))

291.247 sec elapsed


#### 193 km (medium) resolution

In [85]:
tic()
terrain_agg_medium <- purrr::map(terrain_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(terrain_agg_medium, ~write_raster(.x, prefix = "terrain_medium")))

280.117 sec elapsed


## Habitat heterogeneity
Global habitat heterogeneity measures from [earthenv.org](earthenv.org). 2.5 arcmin resolution. These are first and second-order texture indices based on the Enhanced Vegetation Index. The publication for these is [Tuanmu and Jetz 2015](http://onlinelibrary.wiley.com/doi/10.1111/geb.12365/abstract). 

In [88]:
ghh_path <- dir(here("data", "climate_raw", "ghh"), pattern = "*.tif$", full.names = TRUE)
ghh_rast_list <- purrr::map(ghh_path, raster)

#### 96.5 km (high) resolution

In [89]:
tic()
ghh_agg_high <- purrr::map(ghh_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(ghh_agg_high, ~write_raster(.x, prefix = "ghh_high")))

608.204 sec elapsed


#### 193 km (medium) resolution

In [94]:
tic()
ghh_agg_medium <- purrr::map(ghh_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(ghh_agg_medium, ~write_raster(.x, prefix = "ghh_medium")))

571.337 sec elapsed


## Dynamic Habitat Indices
Dynamic Habitat Indices from [the Sylvis Lab](http://silvis.forest.wisc.edu/data/dhis/). 30 arcsecond resolution. There are three vegetation measures: cumulative productivity (band 1), minimum productivity (band 2), and inter-annual variation of productivity (band 3). They are all based on MODIS-derived Enhanced Vegetation Indicies. The publication for this is [Hobi et al. 2017](https://doi.org/10.1016/j.rse.2017.04.018). Since the raw files are kept as a raster stack, I'm using a for loop instead of `purrr::map`.

In [50]:
dhi_path <- dir(here("data", "climate_raw", "dhi"), pattern = "*.tif$", full.names = TRUE)
dhi_rast_list <- stack(dhi_path)

# assign reasonable names to each band
names(dhi_rast_list) <- c("cum", "min", "var")

#### 96.5 km (high) resolution

In [51]:
tic()
for(i in 1:3) {
    dhi_agg_high <- resample_equal_area(dhi_rast_list[[i]], km = res_list$high)
    write_raster(dhi_agg_high, prefix = "dhi_high")
}
toc()

3669.726 sec elapsed


#### 193 km (medium) resolution

In [52]:
tic()
for(i in 1:3) {
    dhi_agg_medium <- resample_equal_area(dhi_rast_list[[i]], km = res_list$medium)
    write_raster(dhi_agg_medium, prefix = "dhi_medium")
}
toc()

3718.069 sec elapsed


## Human modification
0-1 scaled measure of human modification of terrestrial lands across the globe. 30 arc-second resolution.
Citation: Kennedy CM, Oakleaf JR, Theobald DM, Baruch‐Mordo S, Kiesecker J. Managing the middle: A shift in conservation priorities based on the global human modification gradient. Glob Change Biol. 2019;25:811–826. https://doi.org/10.1111/gcb.14549

In [6]:
human_path <- dir(here("data", "climate_raw", "human_mod"), pattern = "*.tif$", full.names = TRUE)
human_rast_list <- raster(human_path)

#### 96.5 km (high) resolution

In [6]:
tic()
human_agg_high <- resample_equal_area(human_rast_list, km = res_list$high)
toc()
# write rasters to file
write_raster(human_agg_high, prefix = "human_high")

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“230 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“230 projected point(s) not finite”


954.678 sec elapsed


#### 193 km (medium) resolution

In [7]:
tic()
human_agg_medium <- resample_equal_area(human_rast_list, km = res_list$medium)
toc()
# write rasters to file
write_raster(human_agg_medium, prefix = "human_medium")

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“230 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“230 projected point(s) not finite”


961.798 sec elapsed
